In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/13 12:14:06 WARN Utils: Your hostname, LAPTOP-F5VFQF4J resolves to a loopback address: 127.0.1.1; using 172.26.218.55 instead (on interface eth0)
25/03/13 12:14:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/13 12:14:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/13 12:14:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df_green = spark.read.parquet('../tmp/data/pq/green/*/*')
df_green.createOrReplaceTempView('green')

In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('../tmp/data/report/revenue/green', mode='overwrite')

25/03/13 12:16:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/13 12:16:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [10]:
df_yellow = spark.read.parquet('../tmp/data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [12]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [13]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('../tmp/data/report/revenue/yellow', mode='overwrite')

25/03/13 12:21:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/13 12:21:11 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

-------------------

In [14]:
df_green_revenue = spark.read.parquet('../tmp/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('../tmp/data/report/revenue/yellow')

In [15]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [16]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [17]:
df_join.write.parquet('../tmp/data/report/revenue/total', mode='overwrite')

25/03/13 12:28:30 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [18]:
df_join = spark.read.parquet('../tmp/data/report/revenue/total')

In [19]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [20]:
df_zones = spark.read.parquet('../tmp/parquet/zones/')

In [21]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [23]:
df_result.drop('LocationID', 'zone').write.parquet('../tmp/parquet/revenue-zones')

25/03/13 12:35:05 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [24]:
df_revenue_zones = spark.read.parquet('../tmp/parquet/revenue-zones')
df_revenue_zones.show(10)

+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|               hour|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------------+--------------------+------------------+---------------------+---------+------------+
|2020-01-01 00:00:00|              NULL|                NULL|1004.3000000000003|                   57|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|              NULL|                NULL|            1214.8|                   56|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|              NULL|                NULL| 732.4800000000002|                   42|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|50.900000000000006|                   3|              NULL|                 NULL|    Bronx|   Boro Zone|
|2020-01-01 00:00:00|              51.9|                   2|              70.8|                    1| Brooklyn|   Bor